In [1]:
from openai import OpenAI
client = OpenAI(base_url='https://thales.xindoo.xyz/openai/v1/')

In [11]:
# 将文件上传至openAI保存
file = client.files.create(
  file=open("flower_prices.csv", "rb"),
  purpose='assistants'
)


In [41]:
file

FileObject(id='file-uhMIBtm4BPXlJlY1UzGIPlGn', bytes=194, created_at=1700398453, filename='flower_prices.csv', object='file', purpose='assistants', status='processed', status_details=None)

In [13]:
# 定义保存账单的方法
def save_bill(totalCost, totalIncome):
    '''保存总成本和总的收入'''
    print(totalCost, totalIncome)
    return "success"

function = {
        "type": "function",
        "function": {
            "name": "save_bill",
            "description": "保存总成本和总的销售额",
            "parameters": {
                "type": "object",
                "properties": {
                    "totalCost": {
                        "type": "number",
                        "description": "总成本",
                    },
                    "totalIncome": {
                        "type": "number",
                        "description": "总收入",
                    }
                },
                "required": ["totalCost", "totalIncome"],
            },
        }
    }
available_functions = { "save_bill": save_bill}  

In [42]:
# 创建助手，将code_interpreter,retrieval,function都开启
assistant = client.beta.assistants.create(
  name="花店财务助手",
  description="按照每种花的售出量，统计成本和收入，计算出总的销售额，给出具体过程和结果",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}, {"type": "retrieval"}, function],
  file_ids=[file.id]
)
assistant

Assistant(id='asst_9sM4lTeCLPjPPYkEGdtzRn8s', created_at=1700404218, description='按照每种花的售出量，统计成本和收入，计算出总的销售额，给出具体过程和结果', file_ids=['file-uhMIBtm4BPXlJlY1UzGIPlGn'], instructions=None, metadata={}, model='gpt-4-1106-preview', name='花店财务助手', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter'), ToolRetrieval(type='retrieval'), ToolFunction(function=FunctionDefinition(name='save_bill', parameters={'type': 'object', 'properties': {'totalCost': {'type': 'number', 'description': '总成本'}, 'totalIncome': {'type': 'number', 'description': '总收入'}}, 'required': ['totalCost', 'totalIncome']}, description='保存总成本和总的收入'), type='function')])

In [43]:
# 创建对话Thread
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "我卖出去了红玫瑰3支、郁金香2支、百合6支，计算下总成本和总收入，给出具体的计算过程"
    }
  ]
)
thread

Thread(id='thread_GjRyMbhhInGmMguYDhH1jxNn', created_at=1700404223, metadata={}, object='thread')

In [44]:
# 创建Run
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
# 从Run中获取结果
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
run

Run(id='run_MANRDjeJYgazCr0mIrH2HE9g', assistant_id='asst_9sM4lTeCLPjPPYkEGdtzRn8s', cancelled_at=None, completed_at=None, created_at=1700404242, expires_at=1700404842, failed_at=None, file_ids=['file-uhMIBtm4BPXlJlY1UzGIPlGn'], instructions=None, last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1700404242, status='in_progress', thread_id='thread_GjRyMbhhInGmMguYDhH1jxNn', tools=[ToolAssistantToolsCode(type='code_interpreter'), ToolAssistantToolsRetrieval(type='retrieval'), ToolAssistantToolsFunction(function=FunctionDefinition(name='save_bill', parameters={'type': 'object', 'properties': {'totalCost': {'type': 'number', 'description': '总成本'}, 'totalIncome': {'type': 'number', 'description': '总收入'}}, 'required': ['totalCost', 'totalIncome']}, description='保存总成本和总的收入'), type='function')])

In [45]:
# client.beta.threads.runs.cancel(run_id=run.id, thread_id=thread.id)
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
run

Run(id='run_MANRDjeJYgazCr0mIrH2HE9g', assistant_id='asst_9sM4lTeCLPjPPYkEGdtzRn8s', cancelled_at=None, completed_at=None, created_at=1700404242, expires_at=1700404842, failed_at=None, file_ids=['file-uhMIBtm4BPXlJlY1UzGIPlGn'], instructions=None, last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1700404242, status='in_progress', thread_id='thread_GjRyMbhhInGmMguYDhH1jxNn', tools=[ToolAssistantToolsCode(type='code_interpreter'), ToolAssistantToolsRetrieval(type='retrieval'), ToolAssistantToolsFunction(function=FunctionDefinition(name='save_bill', parameters={'type': 'object', 'properties': {'totalCost': {'type': 'number', 'description': '总成本'}, 'totalIncome': {'type': 'number', 'description': '总收入'}}, 'required': ['totalCost', 'totalIncome']}, description='保存总成本和总的收入'), type='function')])

In [46]:
if run.status == 'requires_action':
    tool_outputs=[]
    # 调用并保存所有函数调用的结果
    for call in run.required_action.submit_tool_outputs.tool_calls:
        if call.type != "function":
            continue
        # 获取真实函数
        function = available_functions[call.function.name]
        output = {
            "tool_call_id": call.id,
            "output": function(**call.function.arguments),
        }
        tool_outputs.append(output)
    # 将函数调用的结果回传给Assistant
    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

In [ ]:
# run的状态
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
if run.status == 'completed':
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    )
    print(messages)
elif run.status == 'failed':
    print(run.error)

In [40]:
# 添加新消息
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="另外还有2支向日葵，补充下这份账单"
)
# 创建run
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
# 获取执行结果
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)